In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

from datetime import date, timedelta, datetime
from dateutil.relativedelta import relativedelta
from collections import OrderedDict
from scipy import stats
import scorecardpy as sc

from utils import calc_iv, get_top_abs_correlations, performance_table, table_summary, \
                  remove_highcorr_vars_auc, aucs_by_groups



In [ ]:
df = pd.read_csv("dir to data")

feat_set = df.columns.tolist()

# Date Time setting
date_time_var = ['date']

# Variable type setting
int_vars = ['target']

for i in int_vars:
    df[i] = np.where(df[i].isin(['""', '']), np.nan, df[i].astype(int))

float_vars = [i for i in df.columns if i not in date_time_var + int_vars]

for i in float_vars:
    df.loc[df[i].eq('""'),i] = np.nan
    try:
        df[i] = df[i].astype(float)
    except:
        continue

id_col = ['id']
target_col = ['target']

descriptive_col = ['descrip cols']

predictive_cols = [i for i in df.columns if i not in id_col + target_col + descriptive_col]

non_predictive_cols = [i for i in df.columns if i in id_col + target_col + descriptive_col]




In [ ]:
#-----------------------
# Variable Inspection
#-----------------------
"""
check for outliers
check for 0 values
check for infinite values
check for negative values

"""

df['X_fold'] = 'All'
target = 'target'

# heatmap
corr = df[predictive_cols].corr()
sns.heatmap(corr)
plt.show()

lr_features = remove_highcorr_vars_auc(df, predictive_cols, target, 0.75)
get_top_abs_correlations(df[lr_features],10)

df.fillna(0, inplace=True)



In [ ]:
cutoff_date = '----'

oot = df[non_predictive_cols + predictive_cols]
oot = oot[oot['eff_dt']>= cutoff_date].set_index('id')

RANDOM_SEED = 2024
df_model = df[df['date'] < cutoff_date][non_predictive_cols + predictive_cols].set_index('id')

train,test = train_test_split(df_model,
                              test_size = 0.2,
                              random_state= RANDOM_SEED)
train, valid = train_test_split(train, test_size = 0.2, random_state = RANDOM_SEED)

train['X_fold'] = 'train'
test['X_fold'] = 'test'
valid['X_fold'] = 'valid'
oot['X_fold'] = 'oot'

mds = pd.concat([train,test,valid,oot]).copy(deep=True)

mds[target].value_counts()

mds = mds.reset_index()